In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [3]:
f = open("hatha_urls", "rb")
hatha_urls = pickle.load(f)
f.close()

In [4]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Hatha"
    except AttributeError:
        info = {"Title": np.nan}
    except TimeoutError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [5]:
len(hatha_urls)

25628

In [6]:
hatha_classes = [get_poses(url) for url in hatha_urls[11000:13000]]

In [7]:
df = pd.DataFrame(hatha_classes)

In [8]:
f = open("hatha_df", "rb")
hatha_df = pickle.load(f)
f.close()

In [9]:
hatha_df = pd.concat([hatha_df, df])

In [10]:
hatha_df

,Title,Poses,Class Type
0,←Montag abend,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Hatha
1,←Vishuddha (Throat) Balance,"[Easy Pose, Breath Awareness Section, Establis...",Hatha
2,←Breathe yoga,"[Easy Pose, Diaphragmatic Breathing, Seated Ca...",Hatha
3,←___Arm_balance,"[Pranayama, Ocean Breath Variation I Close Up,...",Hatha
4,←floor and hips,"[One Legged Wheel Pose Feet Partner, Flying Wh...",Hatha
...,...,...,...
1995,←October Fall,"[Warmup, Half Bound Thread The Needle Pose, Th...",Hatha
1996,←Crescent moon,"[Easy Pose, Pranayama, Easy Pose Warm Up Flow,...",Hatha
1997,←Sun Salutation - SURYA NAMASKAR A,"[Centering, Mountain Pose Namaste, Mountain Po...",Hatha
1998,←CHAKRA WORKSHOP,"[Easy Pose, Three Part Breath, Easy Pose Warm ...",Hatha


In [11]:
f = open("hatha_df", "wb")
pickle.dump(hatha_df, f)
f.close()